<a href="https://colab.research.google.com/github/shivanigurbani/Analysing-Mental-Health-during-COVID/blob/main/Precovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from string import punctuation

#read train csv
tweet_df = pd.read_csv("Pre Covid.csv", encoding="latin")

In [ ]:
tweet_df.head()

,tweets,labels
0,@AvonteNikole im followin your lil sis she ju...,0
1,@TimmyGrunt &quot;what a waste of eyes&quot;,0
2,fusion brought it tonight! Love my church fam....,0
3,@JackAllTimeLow @AlexAllTimeLow Have an AWESOM...,0
4,@BigBOnTheRoad I say yes! Parking sucks and fo...,0


In [ ]:
ps = PorterStemmer()
negation_list = ["arent","isnt","dont","doesnt","not","cant","couldnt", "werent",
                 "wont","didnt","never","nothing","nowhere","noone","none"
                "hasnt","hadnt","shouldnt","wouldnt","aint"]
               
def preProcessTweets(tweet):
    tweet = tweet.lower()
    tweet = re.sub('n[^A-Za-z ]t','nt', tweet)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)
    tweet = re.sub('@[^\s]+', '', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = word_tokenize(tweet)
    tweet_list = [];
    negate = False
    for word in tweet:
        word = ps.stem(word)
        if word in negation_list:
            negate = True
        elif negate is True and word in list(punctuation):
            negate = False
            
        if negate and word not in negation_list:
            word = "not_"+word
        else:
            pass
        word = re.sub('[^A-Za-z_ ]+', '', word)
        if len(word) > 2 and word not in stopwords.words('english'):
            tweet_list.append(word)
    tweet_set = set(tweet_list)
    return " ".join(tweet_set)

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [ ]:
tweet_df["clean_text"] = tweet_df["tweets"].apply(preProcessTweets)

In [ ]:
tweet_df.to_csv("train_preprocessed.csv")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
import pickle

In [ ]:
tweet_df = pd.read_csv("train_preprocessed.csv")

In [ ]:
tweet_df["clean_text"]=tweet_df["clean_text"].astype('U')

In [ ]:
pipeline = Pipeline([
   ( 'bow',CountVectorizer()),
    ('classifier',MultinomialNB()),
])

In [ ]:
from sklearn.model_selection import train_test_split
msg_train,msg_test,label_train,label_test = train_test_split(tweet_df['clean_text'],tweet_df['labels'],test_size=0.3)

pipeline.fit(msg_train, label_train)

predictions = pipeline.predict(msg_test)


In [ ]:
print(classification_report(predictions, label_test))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1225
           1       0.96      0.89      0.92       369

    accuracy                           0.97      1594
   macro avg       0.96      0.94      0.95      1594
weighted avg       0.97      0.97      0.97      1594

